In [1]:
#头文件
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame
from xml.etree import ElementTree as ET
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D
from keras.layers import *
from datetime import date
from keras.models import *
import tensorflow as tf 

Using TensorFlow backend.


In [2]:
#释放gpu
from numba import cuda
cuda.select_device(0)
cuda.close()

In [3]:
os.environ['KERAS_BACKEND']='tensorflow'
os.environ['CUDA_VISIBLE_DEVICES']="0"
config=tf.compat.v1.ConfigProto() 
config.gpu_options.visible_device_list = '0' 
config.gpu_options.allow_growth = True 
config.gpu_options.per_process_gpu_memory_fraction = 0.7
sess=tf.compat.v1.Session(config=config)

In [4]:
#读取数据
dataset=[]
def data_load():
    path='./traffic-matrices'
    FileList = os.listdir(path)
    for files in FileList:
        oldDirPath = path+'/'+files
        per=ET.parse(oldDirPath)
        p=per.findall('./IntraTM/src')
        matrix=np.zeros((23,23))
        for child in p:
            src=child.attrib['id']
            c=child.findall('dst')
            for i in c:
                dst=i.attrib['id']
                matrix[int(src)-1][int(dst)-1]=i.text
        dataset.append(matrix)
    datasets = np.array(dataset)
    return datasets
datasets = data_load()
datasets.shape

(10773, 23, 23)

In [5]:
#二维流量矩阵转一维
data = []
for i in range(len(datasets)):
    arr = []
    for row in datasets[i]:
        arr.extend(row)
    data.append(arr)
data = np.array(data)
data = data[:,40]

In [6]:
#划分训练集和验证集
time_stamp = 120
train = data[6000:10600+time_stamp].reshape(-1,1)
valid = data[10600-time_stamp:-3].reshape(-1,1)
#归一化
scaler1 = MinMaxScaler(feature_range=(0, 1)) 
#训练集
scaled_data = scaler1.fit_transform(train)
x_train, y_train = [], []
for i in range(time_stamp, len(train)):
    x_train.append(scaled_data[i - time_stamp:i])
    y_train.append(scaled_data[i])
x_train,y_train= np.array(x_train),np.array(y_train)
y_train = y_train.reshape(-1,1)
#验证集
scaled_data2 = scaler1.transform(valid)
x_valid, y_valid = [], []
for i in range(time_stamp, len(valid)):
    x_valid.append(scaled_data2[i - time_stamp:i])
    y_valid.append(scaled_data2[i])
x_valid,y_valid = np.array(x_valid),np.array(y_valid)  
y_valid = y_valid.reshape(-1,1)
print(train.shape,x_train.shape,y_train.shape,x_valid.shape,y_valid.shape)

(4720, 1) (4600, 120, 1) (4600, 1) (170, 120, 1) (170, 1)


In [ ]:
# 超参数
epochs = 100
batch_size = 64
inputs = Input(shape=(time_stamp, 1))
conv = Conv1D(filters=128,kernel_size=2,padding='same',input_shape=(time_stamp,1))(inputs)
pool = MaxPooling1D(pool_size=2,padding='valid')(conv)
conv2 = Conv1D(filters=128,kernel_size=2,padding='same')(pool)
pool2 = MaxPooling1D(pool_size=2,padding='valid')(conv2)
flatten = Flatten()(pool2)
dense = Dense(128)(flatten)
re = RepeatVector(1)(dense)
#lstm_out = Bidirectional(LSTM(lstm_units, activation='relu'), name='bilstm')(x)
lstm_out = LSTM(128, return_sequences=True)(pool2)
drop2 = Dropout(0.5)(lstm_out)
lstm_out2 = LSTM(128)(drop2)
output = Dense(1)(lstm_out2)
model = Model(inputs=inputs, outputs=output)
model.compile(loss='mse', optimizer='adam')
work = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                 validation_data=(x_valid, y_valid), verbose=1)

Train on 4600 samples, validate on 170 samples
Epoch 1/80
4600/4600 [==============================] - 8s 2ms/step - loss: 0.0090 - val_loss: 0.0022
Epoch 2/80
4600/4600 [==============================] - 5s 1ms/step - loss: 0.0024 - val_loss: 0.0017
Epoch 3/80
4600/4600 [==============================] - 6s 1ms/step - loss: 0.0021 - val_loss: 0.0014
Epoch 4/80
4600/4600 [==============================] - 5s 1ms/step - loss: 0.0014 - val_loss: 9.6162e-04
Epoch 5/80
4600/4600 [==============================] - 6s 1ms/step - loss: 0.0012 - val_loss: 7.3131e-04
Epoch 6/80
4600/4600 [==============================] - 7s 2ms/step - loss: 9.9993e-04 - val_loss: 7.9168e-04
Epoch 7/80
4600/4600 [==============================] - 7s 2ms/step - loss: 0.0010 - val_loss: 6.4638e-04
Epoch 8/80
4600/4600 [==============================] - 6s 1ms/step - loss: 0.0010 - val_loss: 0.0013
Epoch 9/80
4600/4600 [==============================] - 8s 2ms/step - loss: 9.7249e-04 - val_loss: 6.3072e-04
Epoch 1

4600/4600 [==============================] - 13s 3ms/step - loss: 6.7248e-04 - val_loss: 5.0217e-04
Epoch 74/80
4600/4600 [==============================] - 13s 3ms/step - loss: 7.1318e-04 - val_loss: 6.3357e-04
Epoch 75/80
4600/4600 [==============================] - 13s 3ms/step - loss: 6.9748e-04 - val_loss: 5.0405e-04
Epoch 76/80
4600/4600 [==============================] - 14s 3ms/step - loss: 6.8675e-04 - val_loss: 6.8299e-04
Epoch 77/80
4480/4600 [============================>.] - ETA: 0s - loss: 6.9043e-04- ETA

In [ ]:
#预测值
y = model.predict(x_valid)
# 反归一化
predict = scaler1.inverse_transform(y)
true = scaler1.inverse_transform(y_valid)
#rmse
rmse= np.sqrt(np.mean(np.power((true - predict), 2)))
print(rmse)
mape = np.mean(np.abs((true - predict) / true)) * 100
print(mape)

In [ ]:
plt.plot(work.history['loss'], label='train', color='b')
plt.legend()
plt.show()
plt.plot(work.history['val_loss'], label='valid',color='r')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
predict = predict.reshape(1,-1)[0]
valid = valid.reshape(1,-1)[0]
plt.plot(np.arange(len(predict)),predict,label='CNN-LSTM',color='r')
plt.plot(np.arange(len(true)),true,label='TRUE',color='b')
plt.legend()
plt.show()

In [ ]:
print(x_train.shape)
import keras
# 查看模型层及参数
model.summary()
# 将模型结构保存为图片
keras.utils.plot_model(model,to_file='cnn-lstm.png', show_shapes=True)


In [ ]:
print(train.shape,x_train.shape,y_train.shape,x_valid.shape,y_valid.shape)